In [1]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path
import os, sys
import numpy as np
from utils import losses
import torch
from utils.utils import *
from scipy.special import softmax
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [7]:
# PSPNet_VOC, UNet_COCO10k
base_path = "./pro_data/UNet_COCO10k/val/"
files = os.listdir(base_path + "output")
cross_entropy_loss_func = losses.CrossEntropyLoss2d(reduction="none")
file_count = len(files)

In [3]:
check_folder_exist(base_path + "loss")
for file in range(file_count):
    output = np_read(base_path + f"output/{file}.npy")
    target = np_read(base_path + f"target/{file}.npy")
    losses = cross_entropy_loss_func(torch.from_numpy(output), torch.from_numpy(target))
    losses = losses.numpy()
    np_write(losses, base_path + f"loss/{file}.npy")
    if (file+1)%100 == 0:
        print(file)

99
199
299
399
499
599
699
799
899
999
1099


In [76]:
file = files[0]
output = np_read(base_path + f"output/{file}")
target = np_read(base_path + f"target/{file}")
image = np_read(base_path + f"image/{file}")
losses = cross_entropy_loss_func(torch.from_numpy(output), torch.from_numpy(target))

In [ ]:
plt.matshow(losses[0])
plt.colorbar()
plt.show()

In [ ]:
plt.matshow(target[0])
plt.show()

In [ ]:
plt.matshow(losses[2])
plt.colorbar()
plt.show()

In [ ]:
plt.matshow(target[2])
plt.show()

In [ ]:
plt.matshow(image[2,1])
plt.show()

In [80]:
output_label = np.argmax(output, axis=1)

In [ ]:
plt.matshow(output_label[2])
plt.show()

## Generate ground truth image-based loss

In [13]:
base_path = "./pro_data/UNet_COCO10k/train/"
files = os.listdir(base_path + "loss")
file_count = len(files)
true_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = np.reshape(losses, (losses.shape[0], -1))
    losses = np.mean(losses, axis=1)
    if true_losses is None:
        true_losses = losses
    else:
        true_losses = np.concatenate((true_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)
np_write(true_losses, base_path + "image_true_losses.npy")

99
199
299
399
499
599
699
799
899
999
1099


## Generate ground truth region-baed loss

In [23]:
## region 8x8
base_path = "./pro_data/UNet_COCO10k/val/"
files = os.listdir(base_path + "output")
file_count = len(files)

In [24]:
import torch
avgpool = torch.nn.AdaptiveAvgPool2d((60,60))
region_based_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)

99


In [25]:
np_write(region_based_losses, base_path + "region_8_8_true_losses.npy")

In [27]:
## region 16x16
import torch
base_path = "./pro_data/UNet_COCO10k/val/"
files = os.listdir(base_path + "output")
file_count = len(files)
avgpool = torch.nn.AdaptiveAvgPool2d((30,30))
region_based_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)
np_write(region_based_losses, base_path + "region_16_16_true_losses.npy")

99


## Entropy part

In [ ]:
# PSPNet_VOC, UNet_COCO10k
base_path = "./pro_data/PSPNet_VOC/train/"
files = os.listdir(base_path + "output")
file_count = len(files)

check_folder_exist(base_path + "entropy")
for file in range(file_count):
    output = np_read(base_path + f"output/{file}.npy")
    output = torch.from_numpy(output)
    output = F.softmax(output, dim=1)
    entropy = torch.sum(torch.mul(-output, torch.log(output + 1e-20)), dim=1).unsqueeze(dim=1)
    np_write(entropy.numpy(), base_path + f"entropy/{file}.npy")
    if (file+1)%100 == 0:
        print(file)

99


## Feature Part

In [3]:
# PSPNet_VOC, UNet_COCO10k
base_path = "./pro_data/UNet_COCO10k/train/"
files = os.listdir(base_path + "output")
file_count = len(files)

check_folder_exist(base_path + "feature")
for file in range(file_count):
    output = np_read(base_path + f"output/{file}.npy")
    output = torch.from_numpy(output)
    output = F.softmax(output, dim=1)
    np_write(output.numpy(), base_path + f"feature/{file}.npy")
    if (file+1)%100 == 0:
        print(file)

99
199
299
399
499
599
699
799
899
999
1099


In [41]:
import torch
file = 17
avgpool = torch.nn.AdaptiveAvgPool2d((60,60))
losses = np_read(base_path + f"loss/{file}.npy")
losses = torch.from_numpy(losses)
losses = avgpool(losses)
losses = losses.reshape(-1)

In [42]:
losses[:10]

tensor([0.0005, 0.0002, 0.0002, 0.0002, 0.0002, 0.0006, 0.0017, 0.0036, 0.0041,
        0.0022])

In [39]:
region_losses = np_read(base_path + "region_8_8_true_losses.npy")

In [43]:
region_losses[losses.shape[0] * file:losses.shape[0] * file+10]

array([0.00047702, 0.00017212, 0.00016255, 0.00016966, 0.00023114,
       0.00060023, 0.00171918, 0.00357157, 0.00408929, 0.00224719],
      dtype=float32)

## Generate 4 splited loss ground truth

In [37]:
base_path = "./pro_data/PSPNet_VOC/val/"
files = os.listdir(base_path + "output")

In [38]:
import torch
avgpool = torch.nn.AdaptiveAvgPool2d((2,2))
region_based_losses = None
for file in files:
    losses = np_read(base_path + f"loss/{file}")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)

In [39]:
np_write(region_based_losses, base_path + "image_split_2_2_true_losses.npy")

In [41]:
region_based_losses.max(), region_based_losses.min()

(1.0793692, 0.0)

## Generate 8x8 patch ground truth for 4 splited image

In [46]:
base_path = "./pro_data/PSPNet_VOC/val/"
files = os.listdir(base_path + "output")

In [47]:
import torch
avgpool = torch.nn.AdaptiveAvgPool2d((60,60))
splited_location_list = [[0,30,0,30], [0,30,30,60],[30,60,0,30],[30,60,30,60]]
region_based_losses = None
for file in files:
    losses = np_read(base_path + f"loss/{file}")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    for i in range(losses.shape[0]):
        for j in range(4):
            location = splited_location_list[j]
            loss = losses[i, location[0]:location[1],location[2]:location[3]]
            loss = loss.reshape(-1)
            if region_based_losses is None:
                region_based_losses = loss
            else:
                region_based_losses = np.concatenate((region_based_losses, loss), axis=0)

In [48]:
np_write(region_based_losses, base_path + "region_8_8_image_split_true_losses.npy")

In [50]:
region_based_losses.shape, region_based_losses.max(), region_based_losses.min()

((5216400,), 10.661509, 0.0)